# Setup

In [1]:
import numpy as np
import scipy.io as sio
import os
import time
import spectral

from sklearn import cluster, metrics
from sklearn.preprocessing import scale
from sklearn.decomposition import PCA
from sklearn.model_selection import train_test_split

from tensorflow import keras

def createImageCubes(X, y, windowSize=5, removeZeroLabels = True):
    margin = int((windowSize - 1) / 2)
    zeroPaddedX = np.zeros((X.shape[0] + 2*margin, X.shape[1] + 2*margin, X.shape[2]))
    zeroPaddedX[margin:X.shape[0] + margin, margin:X.shape[1] + margin, :] = X
    patchesData = np.zeros((X.shape[0] * X.shape[1], windowSize, windowSize, X.shape[2]))
    patchesLabels = np.zeros((X.shape[0] * X.shape[1]))
    patchIndex = 0
    for r in range(margin, zeroPaddedX.shape[0] - margin):
        for c in range(margin, zeroPaddedX.shape[1] - margin):
            patchesData[patchIndex, :, :, :] = zeroPaddedX[r - margin:r + margin + 1, c - margin:c + margin + 1]   
            patchesLabels[patchIndex] = y[r-margin, c-margin]
            patchIndex = patchIndex + 1
    if removeZeroLabels:
        patchesData = patchesData[patchesLabels>0,:,:,:]
        patchesLabels = patchesLabels[patchesLabels>0] - 1
    return patchesData, patchesLabels

def count_FLOPs(model):
    FLOPs = 0
    for i in range(len(model.layers)):
        layer = model.layers[i]
        if layer.__class__.__name__ == 'Conv3D':
            FLOPs += 2*np.prod(layer.kernel_size[:3])*np.prod(layer.output_shape[1:5])
        if layer.__class__.__name__ == 'Conv2D':
            FLOPs += 2*np.prod(layer.kernel_size[:2])*layer.input_shape[3]*np.prod(layer.output_shape[1:4])
        if layer.__class__.__name__ in ('MaxPooling3D', 'AveragePooling3D'):
            FLOPs += np.prod(layer.pool_size[:3])*np.prod(layer.output_shape[1:5])
        if layer.__class__.__name__ in ('MaxPooling2D', 'AveragePooling2D'):
            FLOPs += np.prod(layer.pool_size[:2])*np.prod(layer.output_shape[1:4])
        if layer.__class__.__name__ == 'GlobalAveragePooling3D':
            FLOPs += np.prod(layer.input_shape[1:5])
        if layer.__class__.__name__ == 'GlobalAveragePooling2D':
            FLOPs += np.prod(layer.input_shape[1:4])             
        if layer.__class__.__name__ in ('Activation', 'Add', 'Multiply'):
            FLOPs += np.prod(layer.output_shape[1:])            
        if layer.__class__.__name__ == 'Dense':
            FLOPs += 2*layer.input_shape[1]*layer.output_shape[1]          
    return FLOPs

In [2]:
test_ratio = 0.9
patch = 25
PCsNum = 15
ClusterNum = 16  
HSI = sio.loadmat(os.path.join(os.getcwd(),'data/Indian_pines_corrected.mat'))['indian_pines_corrected']
GT = sio.loadmat(os.path.join(os.getcwd(),'data/Indian_pines_gt.mat'))['indian_pines_gt']
HPCs = PCA(n_components=PCsNum, whiten=True).fit_transform(HSI.reshape(-1, HSI.shape[2])).reshape(HSI.shape[0], HSI.shape[1], PCsNum)

# ClusterCNN

In [3]:
start_time = time.process_time()
clusterstack = np.zeros(HPCs.shape)
cluster_algorithm = 'minikmeans' # default = 'minikmeans'

if cluster_algorithm == 'kmeans':
    k_means = cluster.KMeans(n_clusters=ClusterNum).fit(HPCs.reshape((-1, HPCs.shape[2])))
    clusterlabel = k_means.labels_.reshape((GT.shape))
elif cluster_algorithm == 'spectral':
    specluster = cluster.SpectralClustering(n_clusters=ClusterNum).fit(HPCs.reshape((-1, HPCs.shape[2])))
    clusterlabel = specluster.labels_.reshape((GT.shape))
elif cluster_algorithm == 'density':
    dbcluster = cluster.DBSCAN().fit(HPCs.reshape((-1, HPCs.shape[2])))
    clusterlabel = dbcluster.labels_.reshape((GT.shape))
else:
    mbk_means = cluster.MiniBatchKMeans(n_clusters=ClusterNum).fit(HPCs.reshape((-1, HPCs.shape[2])))
    clusterlabel = mbk_means.labels_.reshape((GT.shape))

for i in range(ClusterNum):
    clusterstack[clusterlabel==i]=np.mean(HPCs[clusterlabel==i], axis=0)
    
timeused = (time.process_time() - start_time)
print('Shape of Clusters:', clusterstack.shape, 'Clustering Time: ', timeused, 's.  # of Clusters:', np.max(clusterlabel)+1)

Shape of Clusters: (145, 145, 15) Clustering Time:  0.921875 s.  # of Clusters: 16


In [4]:
X, y = createImageCubes(clusterstack, GT, windowSize=patch)
Xtrain, Xtest, ytrain, ytest = train_test_split(X, y, test_size=test_ratio, random_state=345,stratify=y)
ytrain = keras.utils.to_categorical(ytrain)

## input layer
IL = keras.Input(shape=(patch, patch, PCsNum), name='IL')
C1 = keras.layers.Conv2D(filters=64, kernel_size=(5,5), activation='relu', name='C1')(IL)
P1 = keras.layers.MaxPool2D()(C1)
C2 = keras.layers.Conv2D(filters=64, kernel_size=(3,3), activation='relu', name='C2')(P1) 
P2 = keras.layers.MaxPool2D()(C2)
## flatten
FL = keras.layers.Flatten(name='FL')(P2)        
## fully connected layers
D1 = keras.layers.Dense(units=128, activation='relu', name='D1')(FL)
D1 = keras.layers.Dropout(0.4, name='dropout1')(D1)
D2 = keras.layers.Dense(units=64, activation='relu', name='D2')(D1)
D2 = keras.layers.Dropout(0.4, name='dropout2')(D2)
OL = keras.layers.Dense(units=ytrain.shape[1], activation='softmax', name='OL')(D2)
        
model = keras.models.Model(inputs=IL, outputs=OL, name='2DCNN_FCs')
        
# compiling the model
adam = keras.optimizers.Adam(lr=0.001, decay=1e-06)
model.compile(loss='categorical_crossentropy', optimizer=adam, metrics=['accuracy'])

model.summary()
print('FLOPs:', format(count_FLOPs(model), ','))

Model: "2DCNN_FCs"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
IL (InputLayer)              [(None, 25, 25, 15)]      0         
_________________________________________________________________
C1 (Conv2D)                  (None, 21, 21, 64)        24064     
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 10, 10, 64)        0         
_________________________________________________________________
C2 (Conv2D)                  (None, 8, 8, 64)          36928     
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 4, 4, 64)          0         
_________________________________________________________________
FL (Flatten)                 (None, 1024)              0         
_________________________________________________________________
D1 (Dense)                   (None, 128)               13

In [5]:
start_time = time.process_time()
# training
model.fit(x=Xtrain, y=ytrain, batch_size=256, epochs=100, verbose=0)
training_time = time.process_time()-start_time

# predict
y_pred = np.argmax(model.predict(Xtest), axis=1)
oa = metrics.accuracy_score(ytest, y_pred)
kappa = metrics.cohen_kappa_score(ytest, y_pred)

print('ClusterCNN:', oa, kappa, ' Training Time:', training_time)

ClusterCNN: 0.9834146341463414 0.981085072060257  Training Time: 25.90625


# ClusterCNN/

In [6]:
start_time = time.process_time()
clusterstack = np.zeros(HPCs.shape)

mbk_means = cluster.MiniBatchKMeans(n_clusters=ClusterNum).fit(HPCs.reshape((-1, HPCs.shape[2])))
clusterlabel = mbk_means.labels_.reshape((GT.shape))

clusterstack = keras.utils.to_categorical(clusterlabel)
    
timeused = (time.process_time() - start_time)
print('Shape of Clusters:', clusterstack.shape, 'Clustering Time: ', timeused, 's.  # of Clusters:', np.max(clusterlabel)+1)

Shape of Clusters: (145, 145, 16) Clustering Time:  0.96875 s.  # of Clusters: 16


In [7]:
X, y = createImageCubes(clusterstack, GT, windowSize=patch)
Xtrain, Xtest, ytrain, ytest = train_test_split(X, y, test_size=test_ratio, random_state=345,stratify=y)
ytrain = keras.utils.to_categorical(ytrain)

## input layer
IL = keras.Input(shape=(patch, patch, ClusterNum), name='IL')
C1 = keras.layers.Conv2D(filters=64, kernel_size=(5,5), activation='relu', name='C1')(IL)
P1 = keras.layers.MaxPool2D()(C1)
C2 = keras.layers.Conv2D(filters=64, kernel_size=(3,3), activation='relu', name='C2')(P1) 
P2 = keras.layers.MaxPool2D()(C2)
## flatten
FL = keras.layers.Flatten(name='FL')(P2)        
## fully connected layers
D1 = keras.layers.Dense(units=128, activation='relu', name='D1')(FL)
D1 = keras.layers.Dropout(0.4, name='dropout1')(D1)
D2 = keras.layers.Dense(units=64, activation='relu', name='D2')(D1)
D2 = keras.layers.Dropout(0.4, name='dropout2')(D2)
OL = keras.layers.Dense(units=ytrain.shape[1], activation='softmax', name='OL')(D2)
        
model = keras.models.Model(inputs=IL, outputs=OL, name='2DCNN_FCs')
        
# compiling the model
adam = keras.optimizers.Adam(lr=0.001, decay=1e-06)
model.compile(loss='categorical_crossentropy', optimizer=adam, metrics=['accuracy'])

start_time = time.process_time()
# training
model.fit(x=Xtrain, y=ytrain, batch_size=256, epochs=100, verbose=0)
training_time = time.process_time()-start_time

# predict
y_pred = np.argmax(model.predict(Xtest), axis=1)
oa = metrics.accuracy_score(ytest, y_pred)
kappa = metrics.cohen_kappa_score(ytest, y_pred)

print('ClusterCNN:', oa, kappa, ' Training Time:', training_time)

ClusterCNN: 0.9805962059620597 0.9778472838954265  Training Time: 25.578125


# UnmixingCNN

Use the abandunt map as the material map. 

In [8]:
from LibUnmixing import unmixing2
# Robust-NMF: https://github.com/neel-dey/robust-nmf
_, coeff, _, _ = unmixing2.robust_nmf(data=(HPCs+6).reshape(-1,PCsNum).T, rank=PCsNum, beta=1.5, reg_val=1, sum_to_one=1)

Initializing rNMF uniformly at random.
Iter = 0; Obj = 2815294.401185665
Iter = 100; Obj = 35241.893964493494; Err = 0.007292507551710248
Iter = 200; Obj = 19512.405911427697; Err = 0.005157332877847633
Iter = 300; Obj = 12508.427410473192; Err = 0.003440948953807533
Iter = 400; Obj = 9521.752953457735; Err = 0.002591761899950171
Iter = 500; Obj = 6957.281545373262; Err = 0.003159029322661715
Iter = 600; Obj = 5620.11624848803; Err = 0.0012524523853722218
Iter = 700; Obj = 5164.666853980262; Err = 0.0005856021940572539
Iter = 800; Obj = 4931.320375207305; Err = 0.0003736167084732865
Iter = 900; Obj = 4775.454110534272; Err = 0.00027891844577636435
Iter = 1000; Obj = 4657.414678951167; Err = 0.0002271705282924718
Maximum number of iterations achieved


In [9]:
X, y = createImageCubes(coeff.T.reshape(HPCs.shape), GT, windowSize=patch)
Xtrain, Xtest, ytrain, ytest = train_test_split(X, y, test_size=test_ratio, random_state=345,stratify=y)
ytrain = keras.utils.to_categorical(ytrain)

## input layer
IL = keras.Input(shape=(patch, patch, PCsNum), name='IL')
C1 = keras.layers.Conv2D(filters=64, kernel_size=(5,5), activation='relu', name='C1')(IL)
P1 = keras.layers.MaxPool2D()(C1)
C2 = keras.layers.Conv2D(filters=64, kernel_size=(3,3), activation='relu', name='C2')(P1) 
P2 = keras.layers.MaxPool2D()(C2)
## flatten
FL = keras.layers.Flatten(name='FL')(P2)        
## fully connected layers
D1 = keras.layers.Dense(units=128, activation='relu', name='D1')(FL)
D1 = keras.layers.Dropout(0.4, name='dropout1')(D1)
D2 = keras.layers.Dense(units=64, activation='relu', name='D2')(D1)
D2 = keras.layers.Dropout(0.4, name='dropout2')(D2)
OL = keras.layers.Dense(units=ytrain.shape[1], activation='softmax', name='OL')(D2)
        
model = keras.models.Model(inputs=IL, outputs=OL, name='2DCNN_FCs')
        
# compiling the model
adam = keras.optimizers.Adam(lr=0.001, decay=1e-06)
model.compile(loss='categorical_crossentropy', optimizer=adam, metrics=['accuracy'])

start_time = time.process_time()
# training
model.fit(x=Xtrain, y=ytrain, batch_size=256, epochs=100, verbose=0)
training_time = time.process_time()-start_time

# predict
y_pred = np.argmax(model.predict(Xtest), axis=1)
oa = metrics.accuracy_score(ytest, y_pred)
kappa = metrics.cohen_kappa_score(ytest, y_pred)

print('ClusterCNN:', oa, kappa, ' Training Time:', training_time)

ClusterCNN: 0.9755013550135502 0.9720432050318235  Training Time: 25.296875


# Only CNN

In [8]:
X, y = createImageCubes(HPCs, GT, windowSize=patch)
Xtrain, Xtest, ytrain, ytest = train_test_split(X, y, test_size=test_ratio, random_state=345,stratify=y)
ytrain = keras.utils.to_categorical(ytrain)

## input layer
IL = keras.Input(shape=(patch, patch, PCsNum), name='IL')
C1 = keras.layers.Conv2D(filters=64, kernel_size=(5,5), activation='relu', name='C1')(IL)
P1 = keras.layers.MaxPool2D()(C1)
C2 = keras.layers.Conv2D(filters=64, kernel_size=(3,3), activation='relu', name='C2')(P1) 
P2 = keras.layers.MaxPool2D()(C2)
## flatten
FL = keras.layers.Flatten(name='FL')(P2)        
## fully connected layers
D1 = keras.layers.Dense(units=128, activation='relu', name='D1')(FL)
D1 = keras.layers.Dropout(0.4, name='dropout1')(D1)
D2 = keras.layers.Dense(units=64, activation='relu', name='D2')(D1)
D2 = keras.layers.Dropout(0.4, name='dropout2')(D2)
OL = keras.layers.Dense(units=ytrain.shape[1], activation='softmax', name='OL')(D2)
        
model1 = keras.models.Model(inputs=IL, outputs=OL, name='2DCNN_FCs')
        
# compiling the model
adam = keras.optimizers.Adam(lr=0.001, decay=1e-06)
model1.compile(loss='categorical_crossentropy', optimizer=adam, metrics=['accuracy'])

for i in range(10):
    start_time = time.process_time()
    # training
    model1.fit(x=Xtrain, y=ytrain, batch_size=256, epochs=100, verbose=0)
    training_time = time.process_time()-start_time

    # predict
    y_pred = np.argmax(model1.predict(Xtest), axis=1)
    oa = metrics.accuracy_score(ytest, y_pred)
    kappa = metrics.cohen_kappa_score(ytest, y_pred)

    print('ClusterCNN:', oa, kappa, ' Training Time:', training_time)

ClusterCNN: 0.9825474254742548 0.9800878989433639  Training Time: 24.578125
ClusterCNN: 0.9842818428184282 0.9820687620898033  Training Time: 22.96875
ClusterCNN: 0.9784281842818429 0.975353664559102  Training Time: 23.84375
ClusterCNN: 0.9823306233062331 0.9798278501007128  Training Time: 23.84375
ClusterCNN: 0.9905691056910569 0.9892419193453392  Training Time: 23.984375
ClusterCNN: 0.9880758807588076 0.9863993907141486  Training Time: 22.9375
ClusterCNN: 0.9867750677506775 0.9849106413530481  Training Time: 22.875
ClusterCNN: 0.984390243902439 0.9821781365285682  Training Time: 23.40625
ClusterCNN: 0.9761517615176152 0.9727802370491969  Training Time: 24.328125
ClusterCNN: 0.9838482384823848 0.9815755713757761  Training Time: 23.734375


# ClusterCNN alter

In [13]:
patch = 7
X, y = createImageCubes(HPCs, GT, windowSize=patch)
Xtrain, Xtest, ytrain, ytest = train_test_split(X, y, test_size=test_ratio, random_state=345,stratify=y)
ytrain = keras.utils.to_categorical(ytrain)

## input layer
IL = keras.Input(shape=(patch, patch, PCsNum), name='IL')
C1 = keras.layers.Conv2D(filters=64, padding='same', kernel_size=(5,5), activation='relu', name='C1')(IL)
#P1 = keras.layers.MaxPool2D()(C1)
C2 = keras.layers.Conv2D(filters=64, padding='same', kernel_size=(3,3), activation='relu', name='C2')(C1) 
#P2 = keras.layers.MaxPool2D()(C2)
## flatten
FL = keras.layers.Flatten(name='FL')(C2)        
## fully connected layers
D1 = keras.layers.Dense(units=128, activation='relu', name='D1')(FL)
D1 = keras.layers.Dropout(0.4, name='dropout1')(D1)
D2 = keras.layers.Dense(units=64, activation='relu', name='D2')(D1)
D2 = keras.layers.Dropout(0.4, name='dropout2')(D2)
OL = keras.layers.Dense(units=ytrain.shape[1], activation='softmax', name='OL')(D2)
        
model1 = keras.models.Model(inputs=IL, outputs=OL, name='2DCNN_FCs') 
model1.summary()

Model: "2DCNN_FCs"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
IL (InputLayer)              [(None, 7, 7, 15)]        0         
_________________________________________________________________
C1 (Conv2D)                  (None, 7, 7, 64)          24064     
_________________________________________________________________
C2 (Conv2D)                  (None, 7, 7, 64)          36928     
_________________________________________________________________
FL (Flatten)                 (None, 3136)              0         
_________________________________________________________________
D1 (Dense)                   (None, 128)               401536    
_________________________________________________________________
dropout1 (Dropout)           (None, 128)               0         
_________________________________________________________________
D2 (Dense)                   (None, 64)                82

In [14]:
# compiling the model
adam = keras.optimizers.Adam(lr=0.001, decay=1e-06)
model1.compile(loss='categorical_crossentropy', optimizer=adam, metrics=['accuracy'])

start_time = time.process_time()
# training
model1.fit(x=Xtrain, y=ytrain, batch_size=256, epochs=100, verbose=0)
training_time = time.process_time()-start_time

# predict
y_pred = np.argmax(model1.predict(Xtest), axis=1)
oa = metrics.accuracy_score(ytest, y_pred)
kappa = metrics.cohen_kappa_score(ytest, y_pred)

print('ClusterCNN:', oa, kappa, ' Training Time:', training_time)

ClusterCNN: 0.944390243902439 0.9364523723897088  Training Time: 6.421875


In [19]:
start_time = time.process_time()
clusterstack = np.zeros(HPCs.shape)

mbk_means = cluster.MiniBatchKMeans(n_clusters=ClusterNum).fit(HPCs.reshape((-1, HPCs.shape[2])))
clusterlabel = mbk_means.labels_.reshape((GT.shape))

clusterstack = keras.utils.to_categorical(clusterlabel)
    
timeused = (time.process_time() - start_time)
print('Shape of Clusters:', clusterstack.shape, 'Clustering Time: ', timeused, 's.  # of Clusters:', np.max(clusterlabel)+1)

Shape of Clusters: (145, 145, 16) Clustering Time:  0.375 s.  # of Clusters: 16


In [21]:
patch = 7
X, y = createImageCubes(clusterstack, GT, windowSize=patch)
Xtrain, Xtest, ytrain, ytest = train_test_split(X, y, test_size=test_ratio, random_state=345,stratify=y)
ytrain = keras.utils.to_categorical(ytrain)

## input layer
IL = keras.Input(shape=(patch, patch, ClusterNum), name='IL')
C1 = keras.layers.Conv2D(filters=64, padding='same', kernel_size=(5,5), activation='relu', name='C1')(IL)
#P1 = keras.layers.MaxPool2D()(C1)
C2 = keras.layers.Conv2D(filters=64, padding='same', kernel_size=(3,3), activation='relu', name='C2')(C1) 
#P2 = keras.layers.MaxPool2D()(C2)
## flatten
FL = keras.layers.Flatten(name='FL')(C2)        
## fully connected layers
D1 = keras.layers.Dense(units=128, activation='relu', name='D1')(FL)
D1 = keras.layers.Dropout(0.4, name='dropout1')(D1)
D2 = keras.layers.Dense(units=64, activation='relu', name='D2')(D1)
D2 = keras.layers.Dropout(0.4, name='dropout2')(D2)
OL = keras.layers.Dense(units=ytrain.shape[1], activation='softmax', name='OL')(D2)
        
model1 = keras.models.Model(inputs=IL, outputs=OL, name='2DCNN_FCs') 
adam = keras.optimizers.Adam(lr=0.001, decay=1e-06)
model1.compile(loss='categorical_crossentropy', optimizer=adam, metrics=['accuracy'])

start_time = time.process_time()
# training
model1.fit(x=Xtrain, y=ytrain, batch_size=256, epochs=100, verbose=0)
training_time = time.process_time()-start_time

# predict
y_pred = np.argmax(model1.predict(Xtest), axis=1)
oa = metrics.accuracy_score(ytest, y_pred)
kappa = metrics.cohen_kappa_score(ytest, y_pred)

print('ClusterCNN:', oa, kappa, ' Training Time:', training_time)

ClusterCNN: 0.767479674796748 0.733618707396763  Training Time: 6.9375
